In [ ]:
#!/usr/bin/env python
# coding: utf-8

from office365.runtime.auth.authentication_context import AuthenticationContext # for microsoft authentication
from office365.sharepoint.client_context import ClientContext # for sharepoint connection
from office365.sharepoint.listitems.caml.caml_query import CamlQuery # currently not being used
from office365.runtime.auth.user_credential import UserCredential # for user-credential authentication
import openpyxl #for reading and modifying excel files
import xlsxwriter #for write,text,numbers,formulas,hyperlink in multiple sheets of excel
import os
from openpyxl import load_workbook #for loading the excel workbook
import json
import pandas as pd

# Sharepoint path initialization
baseurl = 'https://my.metlife.com'
basesite = '/teams/ia/Powerappsdatabase/' # every share point has a home.
siteurl = baseurl + basesite 
#print(siteurl)


#######################################
#Reading data from HCFM_Projects list #
#########################################
sp_list = "HCFM_Projects"
ctx = ClientContext(siteurl).with_credentials(UserCredential("akshi.garg@metlife.com", "March8892")) #credentials to be updated
sp_lists = ctx.web.lists
s_list = sp_lists.get_by_title(sp_list)
l_items = s_list.get_items()
ctx.load(l_items)
ctx.execute_query()

####Fetching the details from HCFM_Projects

# for item in l_items:
#     print(item.properties['Title'],item.properties['ProjectID'],item.properties['ProjectName'], 
#           item.properties['Owner'], item.properties['OwnerEmail'],item.properties['LastSaveDate'],
#           item.properties["Status"],item.properties['ProcessName'], item.properties['ProcessOwner'],
#          item.properties['ProcessStart'], item.properties['ProcessEnd'], item.properties['DocumentedBy'],
#          item.properties['PeriodCovered'], item.properties['WalkthroughDate'], item.properties['WalkthroughProvidedBy'],
#          item.properties['ProcessSummary'], item.properties['CriticalITApplicationsEndUserCom'])



datalist = []
for item in l_items:
    datalist.append({"Title":item.properties['Title'],"Project ID":item.properties['ProjectID'],"Project Name":item.properties['ProjectName'], 
          "Owner":item.properties['Owner'],"Owner Email":item.properties['OwnerEmail'],"Last Save Date":item.properties['LastSaveDate'],
          "Status":item.properties["Status"],"Process Name":item.properties['ProcessName'],"Process Owner": item.properties['ProcessOwner'],
         "Process Start":item.properties['ProcessStart'],"Process End":item.properties['ProcessEnd'],"Documented By": item.properties['DocumentedBy'],
         "Period Covered":item.properties['PeriodCovered'],"Walkthrough Date": item.properties['WalkthroughDate'],"Walkthrough Provided By": item.properties['WalkthroughProvidedBy'],
         "Process Summary":item.properties['ProcessSummary'],"Critical IT Applications End User Computing(EUC) To":item.properties['CriticalITApplicationsEndUserCom'],"Process Number":item.properties['ProcessNumber']})

# print([x.properties for x in l_items])

df2= pd.read_json(json.dumps(datalist))

# Changing formats of columns
df2['Process Owner'] = df2['Process Owner'].astype('str')
df2['Process Start'] = df2['Process Start'].astype('str')
df2['Process End'] = df2['Process End'].astype('str')
df2['Documented By'] = df2['Documented By'].astype('str')
df2['Period Covered'] = df2['Period Covered'].astype('str')
df2['Walkthrough Provided By'] = df2['Walkthrough Provided By'].astype('str')
df2['Process Summary'] = df2['Process Summary'].astype('str')
df2['Critical IT Applications End User Computing(EUC) To'] = df2['Critical IT Applications End User Computing(EUC) To'].astype('str')
df2['Last Save Date']= pd.to_datetime(df2['Last Save Date']).dt.tz_localize(None)
df2['Walkthrough Date']= pd.to_datetime(df2['Walkthrough Date']).dt.tz_localize(None)
df2['Process Number'] = df2['Process Number'].astype('str')

# Replacing null values with space
df3 =df2.replace('nan',"")
#df3.info()

# query_text = """
# <Where>
#         <Eq>
#             <FieldRef Name="Status" />
#             <Values>
#                 <Value Type="Choice">Submitted</Value>
                
#             </Values>
#         </Eq>
# </Where>
# """

#caml_query = CamlQuery.parse(query_text)
#items = s_list.get_items(caml_query)
#ctx.load(items) 
#ctx.execute_query()
#result = s_list.get_items(caml_query).execute_query() 

# Filtering the data from HCFM_Projects where status is Submitted

test=df3.loc[df3['Status'] == "Submitted", 'Project ID']
owner_name = df3.loc[df3['Status'] == "Submitted", 'Owner']
process_name = df3.loc[df3['Status'] == "Submitted", 'Process Name']
process_owner = df3.loc[df3['Status'] == "Submitted", 'Process Owner']
process_start = df3.loc[df3['Status'] == "Submitted", 'Process Start']
process_end = df3.loc[df3['Status'] == "Submitted", 'Process End']
doc_by = df3.loc[df3['Status'] == "Submitted", 'Documented By']
per_cov = df3.loc[df3['Status'] == "Submitted", 'Period Covered']
walk_date = df3.loc[df3['Status'] == "Submitted", 'Walkthrough Date']
walk_prov_by = df3.loc[df3['Status'] == "Submitted", 'Walkthrough Provided By']
process_sum = df3.loc[df3['Status'] == "Submitted", 'Process Summary']
euc = df3.loc[df3['Status'] == "Submitted", 'Critical IT Applications End User Computing(EUC) To']
process_number = df3.loc[df3['Status'] == "Submitted", 'Process Number']

# Creating dataframe with those filterd items
df_temp = {'ID': list(test), 'Owner': list(owner_name), 'Process Name': list(process_name),
           'Process Owner': list(process_owner), 'Process Start': list(process_start), 
           'Process End': list(process_end),'Documented By': list(doc_by), 'Period Covered': list(per_cov),
           'Walkthrough Date': list(walk_date),'Walkthrough Provided By': list(walk_prov_by),
           'Process Summary': list(process_sum),'Critical IT Applications End User Computing(EUC) To': list(euc), 'Process Number': list(process_number)}
test1 = pd.DataFrame(df_temp)
#test1

#######################################
#Reading data from HCFM_Projects Details list #
#########################################
sp_list1 = "HCFM_Project_Details"
ctx = ClientContext(siteurl).with_credentials(UserCredential("akshi.garg@metlife.com", "March8892")) # update credentials for sharepoint list
sp_lists = ctx.web.lists
s_list = sp_lists.get_by_title(sp_list1)
l_items = s_list.get_items()
ctx.load(l_items)
ctx.execute_query()

# Fetching details from HCFM_Project_Details
# datalist1 =[]
# for item in l_items:
#     print(item.properties['Title'],item.properties['Project_x0020_ID'],
#           item.properties['Process_x0020_Step_x0020_ID'],item.properties['Process_x0020_Step_x0020_Descrip'],
#           item.properties['Next_x0020_Step_x0020_ID'],item.properties['Connector_x0020_Label'], item.properties['Shape_x0020_Type'],
#          item.properties['Function'], item.properties['Phase'], item.properties['Control_x0020_ID'],
#          item.properties['End_x0020_Customer'], item.properties['Additional_x0020_Details'], item.properties['Alt_x0020_Description'],
#          item.properties['Step_x0020_Number'])

# print([x.properties for x in l_items])

datalist1 = []
for item in l_items:
    datalist1.append({"Project Name":item.properties['Title'],
                      "Project ID":item.properties['Project_x0020_ID'], 
                      "Process Step ID":item.properties['Process_x0020_Step_x0020_ID'],
                      "Process Step Description":item.properties['Process_x0020_Step_x0020_Descrip'],
                      "Next Step ID":item.properties['Next_x0020_Step_x0020_ID'],
                      "Connector Label":item.properties["Connector_x0020_Label"],
                      "Shape Type":item.properties['Shape_x0020_Type'],
                      "Function": item.properties['Function'],
                      "Phase":item.properties['Phase'],
                      "Control ID":item.properties['Control_x0020_ID'],
                      "End Customer": item.properties['End_x0020_Customer'],
                      "Additional Details":item.properties['Additional_x0020_Details'],
                      "Alt Description": item.properties['Alt_x0020_Description'],
                      "Step Number": item.properties['Step_x0020_Number']})

# Converting into dataframe
df4= pd.read_json(json.dumps((datalist1)))

# Changing the datatypes
df4['Alt Description'] = df4['Alt Description'].astype('str')
df4['Connector Label'] = df4['Connector Label'].astype('str')
df4['End Customer'] = df4['End Customer'].astype('str')

#df4.info()

# Replacing none with space 
df5 =df4.replace('None',"")
df5 =df5.replace('nan',"")
df6 = df5
mask = df6.applymap(lambda x: x is None)
cols = df6.columns[(mask).any()]
for col in df6[cols]:
    df6.loc[mask[col], col] = ''
df6.head()

# Sorting the data with project id and step number
df6=df6.sort_values(by=['Project ID', 'Step Number'])

path = '//Ns-nyamvfs01/aud_data/HCFM Testing/Innovations HCFM Raw Code(Desktop Version)/HCFM Project _ Code/HCFM Project _ Code/Souce Folder/'
#print(path)

#Generating excel files for each project id whose status is Submitted (Creating Process Map tab) here k is the key and v is the dataframe
for k, v in df6.groupby('Project ID'):
    for i in range(len(test1)): #len(test1) gives us the number of rows in dataframe
        if k == test1['ID'][i]:
            f_name = path +"Process_Map_File_" + str(test1['Owner'][i])+ "__" +str(test1['ID'][i])+".xlsx" #need to see this coz running total seems missing
            v.to_excel(f_name, index = False, sheet_name='Process Map')


#for each excel file writing the values in Description tab.
for i in range(len(test1)): #len(test1) gives us the number of rows in dataframe
    f_name = path +"Process_Map_File_" + str(test1['Owner'][i])+ "__" +str(test1['ID'][i])+".xlsx"
    workbook = load_workbook(filename=f_name)
    writer = pd.ExcelWriter(f_name, engine='openpyxl') 
    writer.book = workbook
    sheetname = writer.book.create_sheet('Description')
    sheetname['A1']= str('Process Name:') 
    sheetname['B1']= str(test1.iloc[i,2]) 
    sheetname['C1']= str('Process Owner:') 
    sheetname['D1']= str(test1.iloc[i,3])
    sheetname['A2']= str('Process Start:') 
    sheetname['B2']= str(test1.iloc[i,4]) 
    sheetname['C2']= str('Documented By:') 
    sheetname['D2']= str(test1.iloc[i,6])
    sheetname['A3']= str('Process End:') 
    sheetname['B3']= str(test1.iloc[i,5]) 
    sheetname['C3']= str('Period Covered:') 
    sheetname['D3']= str(test1.iloc[i,7])
    sheetname['A4']= str('Walkthrough Date:') 
    sheetname['B4']= str(test1.iloc[i,8]) 
    sheetname['C4']= str('Walkthrough Provided by:') 
    sheetname['D4']= str(test1.iloc[i,9])
    sheetname['A5']= str('Process Summary:') 
    sheetname['B5']= str(test1.iloc[i,10]) 
    sheetname['A6']= str('Critical IT Applications & End User Computing (EUC) Tools:') 
    sheetname['B6']= str(test1.iloc[i,11])
    sheetname['C5']= str('Process Number:') 
    sheetname['D5']= str(test1.iloc[i,12])
    writer.save()





# dest_path = '//Ns-nyamvfs01/aud_data/HCFM Testing/Innovations HCFM Raw Code(Desktop Version)/HCFM Project _ Code/HCFM Project _ Code/Dest Folder/'

# baseurl = 'https://my.metlife.com'
# basesite = '/teams/ia/Powerappsdatabase/' # every share point has a home.
# siteurl = baseurl + basesite 


# ctx = ClientContext(siteurl).with_credentials(UserCredential("akshi.garg@metlife.com", "March8892"))


# for i in range(len(test1)):
#     f_name1 = dest_path +"Process_Map_File_" + str(test1['Owner'][i])+ "__" +str(test1['ID'][i])+".vsdx"
#     remotepath = "Shared Documents/HCFM/"+ "Process_Map_File_" +  str(test1['Owner'][i])+ "__" +str(test1['ID'][i])+".vsdx" # existing folder path under sharepoint site.

#     with open(f_name1, 'rb') as content_file:
#         file_content = content_file.read()

#     dir, name = os.path.split(remotepath)
#     file = ctx.web.get_folder_by_server_relative_url(dir).upload_file(name, file_content).execute_query()

print("Process Finished Successfully")

#Running the main Python script
#if len(os.listdir(path)) > 0: (from video)
os.system("python Visio_HCFM_Automation_Tool_Server_Integration_Copy.py")
#else : (from video)
#    print("No file to process") (From Video)